# Explore entities relationships

* I use the pip_spacy environment to be able to use plugins and extensions not available in Conda


In [1]:
from importlib import reload
import pandas as pd

import psycopg2
import settings as stt

from itables import init_notebook_mode, show
import re

from time import strftime, gmtime

In [2]:
import postgresql_functions as pgf

In [134]:
# reload(pgf)

## Explore relationships concerning studying

In [191]:
q = """
-- subject verb and sentence with sentence and tokens and complements
with tw1 as (
select  s.pk_sentence, s.sentence, ct2.t_text subject, ct.t_text verb, ct4.t_text compo, ct3.t_text object,
				ct5.t_text prep, ct6.t_text organization, ct7.t_text org1, ct8.t_text org2, ct.fk_mathshistory
from mathshistory.coref_token ct 
	join mathshistory.coref_token ct2 on ct2.t_head_i = ct.t_index 
		and ct2.t_dep_ = 'nsubj'
		--and ct2.t_pos_ = 'PROPN'
		and ct2.fk_sentence = ct.fk_sentence 
	join mathshistory.coref_sentence s on s.pk_sentence = ct.fk_sentence 
	join mathshistory.coref_token ct3  on ct3.fk_sentence = ct.fk_sentence 
			and ct3.t_dep_ = 'dobj'
			and ct3.t_head_i = ct.t_index 
	left join mathshistory.coref_token ct4 on ct4.fk_sentence = ct.fk_sentence 
			and ct4.t_head_i = ct3.t_index 
			and ct4.pk_token < ct3.pk_token 
			and ct4.t_pos_ in ('ADJ', 'NOUN') 
	left join mathshistory.coref_token ct5 on ct5.fk_sentence = ct.fk_sentence 
			and ct5.t_head_i = ct.t_index 
			and ct5.t_text ILIKE 'at' 
	left join mathshistory.coref_token ct6 on ct6.fk_sentence = ct.fk_sentence 
			and ct6.t_head_i = ct5.t_index 	
	left join mathshistory.coref_token ct7 on ct7.fk_sentence = ct.fk_sentence 
			and ct7.t_head_i = ct6.t_index 
	left join mathshistory.coref_token ct8 on ct8.fk_sentence = ct.fk_sentence 
			and ct8.t_head_i = ct7.t_index 
where ct.t_dep_ in ('ROOT','relcl')
and ct.t_pos_  ='VERB'
and ct.t_lemma in ('study') --, 'enter', 'graduate'
and ct.fk_mathshistory > 121
order by s.pk_sentence 
--offset 50
limit 100)
select tw1.fk_mathshistory, concat(tw1.fk_mathshistory, '-', tw1.pk_sentence::varchar, ': ', tw1.sentence) sent, 
	concat_ws(' ', tw1.subject, tw1.verb, tw1.compo, 
		tw1.object, lower(tw1.prep), tw1.organization, tw1.org1, tw1.org2) relation,
    array_agg(array[ct.t_text, ct.t_index::varchar, ct.t_lemma, 
    ct.t_pos_ , ct.t_dep_, ct.t_head_text, t_head_pos_, t_head_i::varchar, ct.t_head_tag_] 
    order by ct.t_index asc)::text token
from tw1, mathshistory.coref_token ct 
where ct.fk_sentence = tw1.pk_sentence
group by tw1.pk_sentence, tw1.sentence, tw1.subject, tw1.verb, tw1.object,
			tw1.compo, tw1.prep, tw1.organization, tw1.fk_mathshistory, tw1.org1, tw1.org2
order by tw1.fk_mathshistory;
"""

In [192]:
result = pgf.sql_explore(q, conn)
print(f'Lines count: {len(result[0])}, errors count: {len(result[1])}, \nFirst lines: {result[0][:1]}')
dfq = pd.DataFrame(result[0])

Lines count: 99, errors count: 0, 
First lines: [(123, '123-8389: Lavanha also studied instruments used in navigation, constructing astrolabes, quadrants and compasses. ', 'Lavanha studied instruments', '{{Lavanha,1200,Lavanha,PROPN,nsubj,studied,VERB,1202,VBD},{also,1201,also,ADV,advmod,studied,VERB,1202,VBD},{studied,1202,study,VERB,ROOT,studied,VERB,1202,VBD},{instruments,1203,instrument,NOUN,dobj,studied,VERB,1202,VBD},{used,1204,use,VERB,acl,instruments,NOUN,1203,NNS},{in,1205,in,ADP,prep,used,VERB,1204,VBN},{navigation,1206,navigation,NOUN,pobj,in,ADP,1205,IN},{",",1207,",",PUNCT,punct,studied,VERB,1202,VBD},{constructing,1208,construct,VERB,advcl,studied,VERB,1202,VBD},{astrolabes,1209,astrolabe,NOUN,dobj,constructing,VERB,1208,VBG},{",",1210,",",PUNCT,punct,astrolabes,NOUN,1209,NNS},{quadrants,1211,quadrant,NOUN,conj,astrolabes,NOUN,1209,NNS},{and,1212,and,CCONJ,cc,quadrants,NOUN,1211,NNS},{compasses,1213,compass,NOUN,conj,quadrants,NOUN,1211,NNS},{.,1214,.,PUNCT,punct,studied,

In [193]:
dfq['new']=dfq[3].apply(lambda x : [i.split(',') for i in (re.sub('{|}}$', '', x)).split('},')])
print(dfq['new'][0])

[['Lavanha', '1200', 'Lavanha', 'PROPN', 'nsubj', 'studied', 'VERB', '1202', 'VBD'], ['also', '1201', 'also', 'ADV', 'advmod', 'studied', 'VERB', '1202', 'VBD'], ['studied', '1202', 'study', 'VERB', 'ROOT', 'studied', 'VERB', '1202', 'VBD'], ['instruments', '1203', 'instrument', 'NOUN', 'dobj', 'studied', 'VERB', '1202', 'VBD'], ['used', '1204', 'use', 'VERB', 'acl', 'instruments', 'NOUN', '1203', 'NNS'], ['in', '1205', 'in', 'ADP', 'prep', 'used', 'VERB', '1204', 'VBN'], ['navigation', '1206', 'navigation', 'NOUN', 'pobj', 'in', 'ADP', '1205', 'IN'], ['"', '"', '1207', '"', '"', 'PUNCT', 'punct', 'studied', 'VERB', '1202', 'VBD'], ['constructing', '1208', 'construct', 'VERB', 'advcl', 'studied', 'VERB', '1202', 'VBD'], ['astrolabes', '1209', 'astrolabe', 'NOUN', 'dobj', 'constructing', 'VERB', '1208', 'VBG'], ['"', '"', '1210', '"', '"', 'PUNCT', 'punct', 'astrolabes', 'NOUN', '1209', 'NNS'], ['quadrants', '1211', 'quadrant', 'NOUN', 'conj', 'astrolabes', 'NOUN', '1209', 'NNS'], ['and

In [194]:
dfq['text'] = dfq['new'].apply(lambda x : [(i[0].upper() + '  (' + ','.join([e.lower() for e in i[1:]]) + ')  ') for i in x])
dfq = dfq[[0,1,2,'text']]

In [195]:
dfq.iloc[0]

0                                                     123
1       123-8389: Lavanha also studied instruments use...
2                             Lavanha studied instruments
text    [LAVANHA  (1200,lavanha,propn,nsubj,studied,ve...
Name: 0, dtype: object

In [196]:
dfq.columns = ['id_text','sentence', 'relation','tokens']
len(dfq)

99

In [197]:
### https://github.com/mwouts/itables/blob/main/docs/advanced_parameters.md
show(dfq.sort_values(by='id_text'), classes="display",

     columnDefs=[{"className": "dt-left", "targets": "_all"}],
     scrollY="500px", scrollCollapse=True, paging=False, column_filters="footer", dom="lrtip")

In [166]:
print(dfq.iloc[12]['relation']), print( dfq.iloc[12]['sentence'])

Nicholas studied liberal arts
55-3052: In Heidelberg Nicholas studied liberal arts, particularly philosophy, for a year before going to the University of Padua in 1417. 


(None, None)

In [67]:
dfq.iloc[12]['tokens']

['IN  (263,in,adp,prep,studied,verb,266,vbd)  ',
 'HEIDELBERG  (264,heidelberg,propn,pobj,in,adp,263,in)  ',
 'NICHOLAS  (265,nicholas,propn,nsubj,studied,verb,266,vbd)  ',
 'STUDIED  (266,study,verb,root,studied,verb,266,vbd)  ',
 'LIBERAL  (267,liberal,adj,amod,arts,noun,268,nns)  ',
 'ARTS  (268,art,noun,dobj,studied,verb,266,vbd)  ',
 '"  (",269,",",punct,punct,arts,noun,268,nns)  ',
 'PARTICULARLY  (270,particularly,adv,advmod,philosophy,noun,271,nn)  ',
 'PHILOSOPHY  (271,philosophy,noun,appos,arts,noun,268,nns)  ',
 '"  (",272,",",punct,punct,studied,verb,266,vbd)  ',
 'FOR  (273,for,adp,prep,studied,verb,266,vbd)  ',
 'A  (274,a,det,det,year,noun,275,nn)  ',
 'YEAR  (275,year,noun,pobj,for,adp,273,in)  ',
 'BEFORE  (276,before,adp,prep,studied,verb,266,vbd)  ',
 'GOING  (277,go,verb,pcomp,before,adp,276,in)  ',
 'TO  (278,to,adp,prep,going,verb,277,vbg)  ',
 'THE  (279,the,det,det,university,propn,280,nnp)  ',
 'UNIVERSITY  (280,university,propn,pobj,to,adp,278,in)  ',
 'OF  (2